In [1]:
%load_ext autotime

In [2]:
import fiona
from shapely.geometry import shape
from sqlalchemy import create_engine

time: 1.03 s


# Input Options

In [3]:
# TABLE options are:
## APPEND -- Add new records to existing table
## CREATE -- Create a new table if it does not already exist
## REPLACE -- Drop table if it exists and create a new table
## TRUNCATE -- Keep table if it exists and truncate existing records

##########
## TODO add "COMPARE -- Compare records to existing table.  Add new records with start dates and close updated reords with end dates
##########

TABLE = "REPLACE"

time: 4 ms


# Load Input Data(shapefile)

In [4]:
# Container to hold data to pass to Oracle Bulk Insert
table_name = ''
fiona_rows = list()
schema = list()

# Load Source Data
with fiona.open(r'BusStops.shp','r') as fiona_layer:
    # Get Table Name
    table_name = fiona_layer.name
    
    # Infer Schema and convert to Oracle datatypes
    for key, value in fiona_layer.schema['properties'].items():
        fiona_type = value.split(':')
        if fiona_type[0] == 'int':
            value = 'NUMBER({0},0)'.format(fiona_type[1])
        elif fiona_type[0] == 'float':
            value = 'NUMBER({0})'.format(fiona_type[1].replace('.',','))
        elif fiona_type[0] =='str':
            value = 'VARCHAR2({0})'.format(fiona_type[1])
        elif fiona_type[0] =='date':
            value = 'DATE'
        
        schema.append((key,value))
    
    # Store Records and convert geometry to WKT
    for feature in fiona_layer:
        properties = feature['properties']
        properties['GEOMETRY'] = shape(feature['geometry']).wkt
        fiona_rows.append(properties)

time: 6.49 s


# Create Oracle Spatial Table

In [9]:
####################################################
##TODO Add Options for Drop and Replace or Append ##
####################################################

#Create Oracle connection and cursor
address = ''
engine = create_engine(address)

#Open Connection
connection = engine.raw_connection()
cursor = connection.cursor()

########################
if TABLE == 'REPLACE':
    #Drop Index if it exists
    cursor.execute("""
        declare
           c int;
        begin
            select count(*) into c from user_sdo_index_info where index_name = upper('{0}_idx');
            if c = 1 then 
                execute immediate 'DROP INDEX {0}_idx';
            end if;
        end;
    """.format(table_name.lower()))
    ##############################################
    # Remove the table from USER_SDO_GEOM_METADATA if it exists
    cursor.execute("""
        declare
           tn VARCHAR2(200) :='{0}';
           c int;
        begin
            select count(*) into c from user_sdo_geom_metadata where table_name = upper('{0}');
            if c = 1 then
                execute immediate 'DELETE FROM user_sdo_geom_metadata WHERE table_name = upper(:tname)' USING tn;
            end if;
        end;
    """.format(table_name.lower()))
    #################################
    # Drop table if it already exists
    cursor.execute("""
        declare
           c int;
        begin
            select count(*) into c from user_tables where table_name = upper('{0}');
            if c = 1 then
                execute immediate 'drop table {0}';
            end if;
        end;
    """.format(table_name.lower()))

if TABLE in ('REPLACE','CREATE'):
    ##############
    # Create Table
    command = """CREATE TABLE {0} (""".format(table_name.lower())
    for field in schema:
        command += """{0} {1},\n""".format(field[0], field[1]) 
    command += """GEOMETRY SDO_GEOMETRY)"""

    cursor.execute(command)

    ###################
    # Set Geometry SRID
    cursor.execute("UPDATE {0} t SET t.geometry.sdo_srid = 3005".format(table_name.lower()))

    ##########################################
    # Register table in USER_SDO_GEOM_METADATA
    cursor.execute("INSERT INTO user_sdo_geom_metadata (table_name,column_name,diminfo,srid) VALUES ('{0}','GEOMETRY', sdo_dim_array(sdo_dim_element('X',395388.99,1854357.76,0.005),sdo_dim_element('Y',369155.12,1714756.85,0.005)), 3005)".format(table_name.upper()))

    #####################
    # Build spatial index
    cursor.execute("CREATE INDEX {0}_idx ON {0}(geometry) INDEXTYPE IS MDSYS.SPATIAL_INDEX".format(table_name).lower())
    connection.commit()

############################
#close cursor and connection
cursor.close()
connection.close()

time: 1.36 s


# Insert Data

In [10]:
# Build SQL statement
command = """INSERT INTO {0} (\n""".format(table_name.lower())
for field in schema:
    command += '{0},\n'.format(field[0])
command += 'GEOMETRY)\n'
command += 'VALUES (\n'
for field in schema:
    if field[1] == 'DATE':
        command += "TO_DATE(:{0},'YYYY-MM-DD'),\n".format(field[0])
    else:
        command += ':{0},\n'.format(field[0])
command += 'SDO_GEOMETRY(:GEOMETRY,3005))'

#Open Connection
connection = engine.raw_connection()
cursor = connection.cursor()

# Bulk Load
cursor.executemany(command, fiona_rows)
connection.commit()

#close cursor and connection
cursor.close()
connection.close()

time: 11.1 s
